In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Imports for drawing images

In [ ]:
import pandas as pd
import cv2
from ast import literal_eval
import os
from os.path import isfile, join

Imports for CNN

In [ ]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
import numpy as np
import shutil

# Create Image Data From Raw CSV Data

In [ ]:
BASE_SIZE = 256
size = 80

In [ ]:
def draw_cv2(raw_strokes, size=256, lw=6):
    """
    Takes in one array of the vectors from drawing and returns an image array
    """
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for stroke in raw_strokes:
        for i in range(len(stroke[0]) - 1):
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]), (stroke[0][i + 1], stroke[1][i + 1]), 255, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

In [ ]:
def create_images(filename):
    """
    Takes the filename of one category and creates train and valid folders with all images for that category
    """
    folder = filename.split('.')[0]
    if not os.path.isdir("./train/" + folder):
        os.mkdir("train/" + folder)
    if not os.path.isdir("./valid/" + folder):
        os.mkdir("valid/" + folder)
    df = pd.read_csv(filename)
    df['drawing'] = df['drawing'].apply(literal_eval)
    for i, raw_strokes in enumerate(df.drawing.values):
        if i % 5 == 0:
            cv2.imwrite(f'valid/{folder}/{i}.jpg', draw_cv2(raw_strokes, size=size))
        else:
            cv2.imwrite(f'train/{folder}/{i}.jpg', draw_cv2(raw_strokes, size=size))

In [ ]:
def create_all_images():
    """
    Creates images for all categories
    """
    if not os.path.isdir("./train"):
        os.mkdir("train")
    if not os.path.isdir("./valid"):
        os.mkdir("valid")
    training_set = './../train_simplified/'
    train_files = [join(training_set, f) for f in os.listdir(training_set) if isfile(join(training_set, f))]
    for filename in train_files:
        filename = filename.split("/")[-1]
        create_images(filename)

In [ ]:
create_all_images()

# Create and Run CNN on Image Data

In [ ]:
arch=resnet50
